# Cleaning: BJH049


This script cleans the data both by abnormal trials and by noisy channels and epileptic activity. 


### Prep

In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
#import mat73
import re
#from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
#from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
#matplotlib qt5

In [2]:
# helper functions#

def rle(inarray):
        """ run length encoding. Partial credit to R rle function. 
            Multi datatype arrays catered for including non Numpy
            returns: tuple (runlengths, startpositions, values) """
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])
        
def find_bad_trial_times(idx, srate, trial_begins, trial_lengths):
    ''' takes and index of trial lengths, beginnings and the sampling rate and returns the the start time and and
    duration of the times to annotae as BAD '''
    start_time = trial_begins[0][idx]/srate
    duration = trial_lengths[0][idx]/srate
    
    return start_time, duration        

In [3]:
## Prep paths ##

subject = 'BJH049'
raw_data_dir = f"/home/knight/barbarab/pacman_ieeg/raw_data/{subject}/ieeg"
preproc_data_dir = f"/home/knight/barbarab/pacman_ieeg/preprocessing/{subject}/ieeg"

In [4]:
## Load Data ##

raw_fif = mne.io.Raw(f"{raw_data_dir}/{subject}_raw_ieeg.fif")

Opening raw data file /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_raw_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_raw_ieeg-1.fif...
    Range : 2086000 ... 3354399 =   1043.000 ...  1677.199 secs
Ready.


In [5]:
## Filtering ##

# load data #
raw_fif.load_data()

# highpass filter @ 1 and lowpass @ 200 #
filtered_data = raw_fif.filter(l_freq=1, h_freq=150) # skip this step for TF analysis, always do this on the least processed data, or do .1 to 250

# remove 60hz line noise #
freqs = (60, 120, 180, 240)
filtered_notch_data = filtered_data.notch_filter(freqs=freqs) 


Reading 0 ... 3354399  =      0.000 ...  1677.199 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 150.00 Hz
- Upper transition bandwidth: 37.50 Hz (-6 dB cutoff frequency: 168.75 Hz)
- Filter length: 6601 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    4.6s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:   20.7s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   48.0s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13201 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.4s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:   14.2s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:   32.1s


In [6]:
## Load/Save Filtered Data ##

# save #
filtered_notch_data.save(f"{raw_data_dir}/{subject}_notched_filtered_ieeg.fif", overwrite = True)

# load #
filtered_notch_data = mne.io.Raw(f"{raw_data_dir}/{subject}_notched_filtered_ieeg.fif")
events = mne.find_events(filtered_notch_data, output='step', consecutive = False, stim_channel='STI')

Overwriting existing file.
Writing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_ieeg.fif
Closing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_ieeg.fif
Overwriting existing file.
Writing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_ieeg-1.fif
Closing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_ieeg-1.fif
[done]
Opening raw data file /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_ieeg-1.fif...
    Range : 2086000 ... 3354399 =   1043.000 ...  1677.199 secs
Ready.
480 events found on stim channel STI
Event IDs: [0 1]


In [7]:
filtered_notch_data.info['ch_names']
print(raw_fif.info['ch_names'])


[' AR1', 'AR2', 'AR3', 'AR4', 'REF1', 'REF2', 'AR5', 'AR6', 'AR7', 'AR8', 'AR9', 'AR10', 'AR11', 'AR12', 'AR13', 'AR14', 'AR15', 'BR1', 'BR2', 'BR3', 'BR4', 'BR5', 'BR6', 'BR7', 'BR8', 'CR1', 'CR2', 'CR3', 'CR4', 'CR5', 'CR6', 'CR7', 'CR8', 'CR9', 'CR10', 'CR11', 'CR12', 'DR1', 'DR2', 'DR3', 'DR4', 'DR5', 'DR6', 'DR7', 'DR8', 'ER1', 'ER2', 'ER3', 'ER4', 'ER5', 'ER6', 'ER7', 'ER8', 'FR1', 'FR2', 'FR3', 'FR4', 'FR5', 'FR6', 'FR7', 'FR8', 'FR9', 'FR10', 'FR11', 'FR12', 'GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6', 'GR7', 'GR8', 'HR1', 'HR2', 'HR3', 'HR4', 'HR5', 'HR6', 'HR7', 'HR8', 'HR9', 'HR10', 'HR11', 'HR12', 'HR13', 'HR14', 'HR15', 'IR1', 'IR2', 'IR3', 'IR4', 'IR5', 'IR6', 'IR7', 'IR8', 'IR9', 'IR10', 'JR1', 'JR2', 'JR3', 'JR4', 'JR5', 'JR6', 'JR7', 'JR8', 'JR9', 'JR10', 'JR11', 'JR12', 'JR13', 'JR14', 'JR15', 'JR16', 'JR17', 'JR18', 'KR1', 'KR2', 'KR3', 'KR4', 'KR5', 'KR6', 'KR7', 'KR8', 'KR9', 'KR10', 'LR1', 'LR2', 'LR3', 'LR4', 'LR5', 'LR6', 'LR7', 'LR8', 'LR9', 'LR10', 'LR11', 'LR12'

## Cleaning Neural Data: BJH049

DID NOT USE -- TOO MESSY

#### Bad Channels

Epileptic channels from my cleaning:

Noisy channels:['BL1','BL2','BL3','BL4','BL5','BL14','BL15',' AR1','AR2','AR3','AR4','BR1','BR2','BR3','BR8','ER1','ER2','ER3','ER4','ER5','ER6','ER7','ER8','CR1','CR2','CR3','CR4','CR7','CR8','CR9','CR10','CR11','DR1','DR2','DR3','DR4','DR5','DR6','DR7','DR8','HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8','IR1','KR1','KR2']


In [15]:
# Remove Channels from filtered data

# remove non seeg electrodes electrodes #
string_lst = ['EMPTY', 'REF', 'ekg']

bad_channels_reg = [re.findall(r"(?=("+'|'.join(string_lst)+r"))", name, re.IGNORECASE) for name in filtered_notch_data.info['ch_names']]
bad_channels_index = [i for i,x in enumerate(bad_channels_reg) if  x]
bad_channels = [filtered_notch_data.info['ch_names'][i] for i in bad_channels_index]

filtered_notch_data.info['bads'].extend(bad_channels)  

# # remove epileptic electrodes --- uncomment these 3 lines after inspection with BOB
bob_bad_channels =['BL1','BL2','BL3','BL4','BL5','BL14','BL15',' AR1','AR2','AR3','AR4','BR1','BR2','BR3','BR8','ER1','ER2','ER3','ER4','ER5','ER6','ER7','ER8','CR1','CR2','CR3','CR4','CR7','CR8','CR9','CR10','CR11','DR1','DR2','DR3','DR4','DR5','DR6','DR7','DR8','HR1','HR2','HR3','HR4','HR5','HR6','HR7','HR8','IR1','KR1','KR2']
filtered_notch_data.info['bads'].extend(bob_bad_channels)  


# remove out of brain electrodes
## TODO hasn't been localized

# remove scalp and dc channels
scalp_dc_channels = ['FP1', 'F3', 'C3', 'P3', 'O1', 'FP2', 'F4', 'C4', 'P4', 'O2', 'F7', 'T7', 'P7', 'F8', 'T8', 'P8', 'F9', 'F10', 'FPZ', 'FZ', 'CZ', 'PZ', 'OZ']


filtered_notch_data.info['bads'].extend(scalp_dc_channels)
filtered_notch_data

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,0 points
Good channels,"131 sEEG, 1 Stimulus"
Bad channels,"REF1, REF2, EMPTY, EMPTY_186, EMPTY_187, EMPTY_188, EMPTY_189, EMPTY_190, EMPTY_191, EMPTY_192, EMPTY_193, EMPTY_194, EMPTY_195, EMPTY_196, EMPTY_197, EMPTY_198, EMPTY_199, EMPTY_200, EMPTY_201, EMPTY_202, EMPTY_203, EMPTY_204, EMPTY_205, EMPTY_206, EMPTY_207, EMPTY_208, EMPTY_209, EMPTY_210, EMPTY_211, EMPTY_212, EMPTY_213, EMPTY_214, EMPTY_215, EMPTY_216, EMPTY_217, EMPTY_218, EMPTY_219, EMPTY_220, EMPTY_221, EMPTY_222, EMPTY_223, EMPTY_224, EMPTY_225, EMPTY_226, EMPTY_227, EMPTY_228, EMPTY_229, EMPTY_230, EMPTY_231, EKG1, EKG2, BL1, BL2, BL3, BL4, BL5, BL14, BL15, AR1, AR2, AR3, AR4, BR3, BR8, FP1, F3, C3, P3, O1, FP2, F4, C4, P4, O2, F7, T7, P7, F8, T8, P8, F9, F10, FPZ, FZ, CZ, PZ, OZ, REF1, REF2, EMPTY, EMPTY_186, EMPTY_187, EMPTY_188, EMPTY_189, EMPTY_190, EMPTY_191, EMPTY_192, EMPTY_193, EMPTY_194, EMPTY_195, EMPTY_196, EMPTY_197, EMPTY_198, EMPTY_199, EMPTY_200, EMPTY_201, EMPTY_202, EMPTY_203, EMPTY_204, EMPTY_205, EMPTY_206, EMPTY_207, EMPTY_208, EMPTY_209, EMPTY_210, EMPTY_211, EMPTY_212, EMPTY_213, EMPTY_214, EMPTY_215, EMPTY_216, EMPTY_217, EMPTY_218, EMPTY_219, EMPTY_220, EMPTY_221, EMPTY_222, EMPTY_223, EMPTY_224, EMPTY_225, EMPTY_226, EMPTY_227, EMPTY_228, EMPTY_229, EMPTY_230, EMPTY_231, EKG1, EKG2, REF1, REF2, EMPTY, EMPTY_186, EMPTY_187, EMPTY_188, EMPTY_189, EMPTY_190, EMPTY_191, EMPTY_192, EMPTY_193, EMPTY_194, EMPTY_195, EMPTY_196, EMPTY_197, EMPTY_198, EMPTY_199, EMPTY_200, EMPTY_201, EMPTY_202, EMPTY_203, EMPTY_204, EMPTY_205, EMPTY_206, EMPTY_207, EMPTY_208, EMPTY_209, EMPTY_210, EMPTY_211, EMPTY_212, EMPTY_213, EMPTY_214, EMPTY_215, EMPTY_216, EMPTY_217, EMPTY_218, EMPTY_219, EMPTY_220, EMPTY_221, EMPTY_222, EMPTY_223, EMPTY_224, EMPTY_225, EMPTY_226, EMPTY_227, EMPTY_228, EMPTY_229, EMPTY_230, EMPTY_231, EKG1, EKG2, REF1, REF2, EMPTY, EMPTY_186, EMPTY_187, EMPTY_188, EMPTY_189, EMPTY_190, EMPTY_191, EMPTY_192, EMPTY_193, EMPTY_194, EMPTY_195, EMPTY_196, EMPTY_197, EMPTY_198, EMPTY_199, EMPTY_200, EMPTY_201, EMPTY_202, EMPTY_203, EMPTY_204, EMPTY_205, EMPTY_206, EMPTY_207, EMPTY_208, EMPTY_209, EMPTY_210, EMPTY_211, EMPTY_212, EMPTY_213, EMPTY_214, EMPTY_215, EMPTY_216, EMPTY_217, EMPTY_218, EMPTY_219, EMPTY_220, EMPTY_221, EMPTY_222, EMPTY_223, EMPTY_224, EMPTY_225, EMPTY_226, EMPTY_227, EMPTY_228, EMPTY_229, EMPTY_230, EMPTY_231, EKG1, EKG2, REF1, REF2, EMPTY, EMPTY_186, EMPTY_187, EMPTY_188, EMPTY_189, EMPTY_190, EMPTY_191, EMPTY_192, EMPTY_193, EMPTY_194, EMPTY_195, EMPTY_196, EMPTY_197, EMPTY_198, EMPTY_199, EMPTY_200, EMPTY_201, EMPTY_202, EMPTY_203, EMPTY_204, EMPTY_205, EMPTY_206, EMPTY_207, EMPTY_208, EMPTY_209, EMPTY_210, EMPTY_211, EMPTY_212, EMPTY_213, EMPTY_214, EMPTY_215, EMPTY_216, EMPTY_217, EMPTY_218, EMPTY_219, EMPTY_220, EMPTY_221, EMPTY_222, EMPTY_223, EMPTY_224, EMPTY_225, EMPTY_226, EMPTY_227, EMPTY_228, EMPTY_229, EMPTY_230, EMPTY_231, EKG1, EKG2, BL1, BL2, BL3, BL4, BL5, BL14, BL15, AR1, AR2, AR3, AR4, BR1, BR2, BR3, BR8, ER1, ER2, ER3, ER4, ER5, ER6, ER7, ER8, CR1, CR2, CR3, CR4, CR7, CR8, CR9, CR10, CR11, DR1, DR2, DR3, DR4, DR5, DR6, DR7, DR8, HR1, HR2, HR3, HR4, HR5, HR6, HR7, HR8, IR1, KR1, KR2, FP1, F3, C3, P3, O1, FP2, F4, C4, P4, O2, F7, T7, P7, F8, T8, P8, F9, F10, FPZ, FZ, CZ, PZ, OZ"
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2000.00 Hz
Highpass,1.00 Hz
Lowpass,150.00 Hz


In [9]:
# Remove Channels from filtered data

# remove non seeg electrodes electrodes #
string_lst = ['EMPTY', 'REF', 'ekg']

bad_channels_reg = [re.findall(r"(?=("+'|'.join(string_lst)+r"))", name, re.IGNORECASE) for name in raw_fif.info['ch_names']]
bad_channels_index = [i for i,x in enumerate(bad_channels_reg) if  x]
bad_channels = [raw_fif.info['ch_names'][i] for i in bad_channels_index]

raw_fif.info['bads'].extend(bad_channels)  

# # remove epileptic electrodes
# # remove epileptic electrodes
#filtered_notch_data.info['bads'].append('')  
#filtered_notch_data.info['bads'].append('')  
#filtered_notch_data.info['bads'].append('')  

# remove out of brain electrodes
## TODO hasn't been localized

# remove scalp and dc channels
scalp_dc_channels = []

raw_fif.info['bads'].extend(scalp_dc_channels)
raw_fif

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,0 points
Good channels,"205 sEEG, 1 Stimulus"
Bad channels,"REF1, REF2, EMPTY, EMPTY_186, EMPTY_187, EMPTY_188, EMPTY_189, EMPTY_190, EMPTY_191, EMPTY_192, EMPTY_193, EMPTY_194, EMPTY_195, EMPTY_196, EMPTY_197, EMPTY_198, EMPTY_199, EMPTY_200, EMPTY_201, EMPTY_202, EMPTY_203, EMPTY_204, EMPTY_205, EMPTY_206, EMPTY_207, EMPTY_208, EMPTY_209, EMPTY_210, EMPTY_211, EMPTY_212, EMPTY_213, EMPTY_214, EMPTY_215, EMPTY_216, EMPTY_217, EMPTY_218, EMPTY_219, EMPTY_220, EMPTY_221, EMPTY_222, EMPTY_223, EMPTY_224, EMPTY_225, EMPTY_226, EMPTY_227, EMPTY_228, EMPTY_229, EMPTY_230, EMPTY_231, EKG1, EKG2"
EOG channels,Not available
ECG channels,Not available
Sampling frequency,2000.00 Hz
Highpass,1.00 Hz
Lowpass,150.00 Hz


#### Bad Epochs from noisy data



In [16]:
# Annotate Noisy Epochs from Bob's cleaning

# read from csv from interactive session
bad_annots = mne.read_annotations('../ieeg/saved_annotations.fif')   


#### Bad epochs from bad trial data

In [ ]:
# ## quality check trials ##

# # calculate trial onsets and offsets
# sti_raw = filtered_notch_data.get_data(picks = ['STI'])
# lengths, positionsm, val  = rle(sti_raw[0])

# # lengths, beginnings, endings, for trials
# trial_lengths = [lengths[x] for x in np.where(val == 1)]
# trial_begins = [positionsm[x] for x in np.where(val == 1)]
# trial_ends = trial_lengths[0] + trial_begins[0]
# trial_baseline = trial_begins[0] - 1000

# # lengths, beginnings, endings, for itis
# iti_lengths = [lengths[x] for x in np.where(val == 0)]
# iti_begins = [positionsm[x] for x in np.where(val == 0)]
# iti_ends = iti_begins[0] + iti_lengths[0]

# # save trial lengths for later
# np.save(f"{preproc_data_dir}/{subject}_trial_lengths.npy", trial_lengths)

In [ ]:
# # exclude bad trials/apochs from r behavioral analysis (paused trials and no biscuits)

# # load bad trial data
# bad_trials = np.genfromtxt(f"{raw_data_dir}/behave/{subject}_bad_trials.csv", delimiter = ',', skip_header = 1)

# onsets = []
# durations = []
# for bad in bad_trials:
#     start, dur = find_bad_trial_times(int(bad), raw_fif.info['sfreq'], trial_begins, trial_lengths)
#     onsets.append(start)
#     durations.append(dur)

# # update descriptions
# descriptions = ['bad'] * len(durations)


In [ ]:
## exclude all bad epochs from both bad trials and investigating neural data ##

# # combine with bad trial data #
# bad_annots.append(onsets, durations, descriptions)

# # # apply to filtered data #
# filtered_notch_data.set_annotations(bad_annots)

# # # apply to not filtered data #
# raw_fif.set_annotations(bad_annots)

## Visualizing Neural Data

In [9]:
# plot for cleaning #

filtered_notch_data.plot(events=events, color='b', bad_color = 'red', n_channels = 230, clipping = None, event_color = 'r')

Using qt as 2D backend.


qt.qpa.xcb: X server does not support XInput 2
Qt: Session management error: Authentication Rejected, reason : None of the authentication protocols specified are supported and host-based authentication failed
qt.qpa.xcb: QXcbConnection: XCB error: 1 (BadRequest), sequence: 169, resource id: 132, major code: 130 (Unknown), minor code: 47


qt.qpa.xcb: QXcbConnection: XCB error: 3 (BadWindow), sequence: 3528, resource id: 6380031, major code: 40 (TranslateCoords), minor code: 0


In [10]:
## From interactive sessions --- saving annotations from BOB
interactive_annot = filtered_notch_data.annotations
for x in range(0, len(filtered_notch_data.annotations)):
    print(interactive_annot[x])
    
    
filtered_notch_data.annotations.save('../ieeg/saved_annotations.fif', overwrite = True) 

OrderedDict([('onset', 17.834512457705323), ('duration', 1.7998857494397313), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 77.95403612075407), ('duration', 0.11952366304873863), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 152.20283868699744), ('duration', 0.7733884079623863), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 158.42509996924025), ('duration', 0.8436964450498863), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 183.48244496198973), ('duration', 0.8577580524673749), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 193.63305796018807), ('duration', 0.49215625961244314), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 242.78233071142947), ('duration', 2.587335764819642), ('description', 'bad'), ('orig_time', None)])
OrderedDict([('onset', 251.17227666212597), ('duration', 2.5873357648196134), ('description', 'bad'), ('orig_time', None)])
OrderedD

/tmp/ipykernel_1990570/3068101005.py:7: RuntimeWarning: This filename (../ieeg/saved_annotations.fif) does not conform to MNE naming conventions. All annotations files should end with -annot.fif, -annot.fif.gz, _annot.fif, _annot.fif.gz, .txt or .csv
  filtered_notch_data.annotations.save('../ieeg/saved_annotations.fif', overwrite = True)


In [29]:
# plot psd of all channels #

# filtered_notch_data.plot_psd()

In [ ]:
# plot psd of bad channels #

# filtered_notch_data.plot_psd(picks = filtered_notch_data.info['bads'])

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().


ValueError: Found 85 / 207 unique names, sel is not unique

## Save Files

In [17]:
# save filtered data #
filtered_notch_data.save(f"{raw_data_dir}/{subject}_notched_filtered_clean_ieeg.fif", overwrite = True)


# save raw clean data #
raw_fif.save(f"{raw_data_dir}/{subject}_raw_clean_ieeg.fif", overwrite = True)


Overwriting existing file.
Writing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_clean_ieeg.fif


Closing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_clean_ieeg.fif
Overwriting existing file.
Writing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_clean_ieeg-1.fif
Closing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_notched_filtered_clean_ieeg-1.fif
[done]
Overwriting existing file.
Writing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_raw_clean_ieeg.fif
Closing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_raw_clean_ieeg.fif
Overwriting existing file.
Writing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_raw_clean_ieeg-1.fif
Closing /home/knight/barbarab/pacman_ieeg/raw_data/BJH049/ieeg/BJH049_raw_clean_ieeg-1.fif
[done]
